In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q datasets huggingface_hub
!huggingface-cli login

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens

In [ ]:
from datasets import load_dataset
import pandas as pd

# Load dataset and save the dataset in 'pattern' and 'response' format
def process_dataset(output_path="/content/drive/MyDrive/converted_dataset.csv"):
    dataset = load_dataset("victunes/nart-100k-synthetic-buddy-mixed-names", split="train")

    converted_data = []
    for row in dataset:
        if 'conversations' in row:
            conversation = row['conversations']
            for i in range(0, len(conversation) - 1, 2):
                user_msg = conversation[i]['value']
                assistant_msg = conversation[i + 1]['value']
                converted_data.append({
                    "pattern": user_msg,
                    "response": assistant_msg
                })

    # Convert to DataFrame and save to CSV
    df = pd.DataFrame(converted_data)
    df.to_csv(output_path, index=False, encoding="utf-8-sig")

    print(f"Dataset successfully saved in 'pattern' and 'response' format at: {output_path}")

# Execute the function
process_dataset("/content/drive/MyDrive/converted_dataset.csv")



Dataset successfully saved in 'pattern' and 'response' format at: /content/drive/MyDrive/converted_dataset.csv


In [ ]:
csv_file = "/content/drive/MyDrive/converted_dataset.csv"
df = pd.read_csv(csv_file)

df.head()
print(len(df))

807085


In [ ]:
# last 4 lakh data

import pandas as pd
from transformers import pipeline
import math

# ✅ Load the dataset
df = pd.read_csv("/content/drive/MyDrive/converted_dataset.csv")

# ✅ Select the last 400,000 rows
df_last_chunk = df.head(407085).copy()

# ✅ Load the emotion classifier model
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=1)

# ✅ Function to split text into 512-token chunks
def split_text(text, max_tokens=512):
    words = text.split()  # Split by whitespace
    return [' '.join(words[i:i + max_tokens]) for i in range(0, len(words), max_tokens)]

# ✅ Apply emotion classification with chunk handling
def get_emotion(text):
    # If text length is greater than 512 tokens, split it
    chunks = split_text(text, max_tokens=512)
    emotions = []

    # Classify each chunk and store the predicted emotion
    for chunk in chunks:
        emotion = emotion_classifier(chunk)
        emotions.append(emotion[0][0]['label'])

    # Aggregate emotions: pick the most frequent emotion
    return max(set(emotions), key=emotions.count)

# ✅ Apply the function to the 'pattern' column in the last chunk
df_last_chunk['tag'] = df_last_chunk['pattern'].apply(get_emotion)

# ✅ Save the updated last chunk to a new CSV file
df_last_chunk.to_csv("/content/drive/MyDrive/converted_dataset_first_400k_with_tags.csv", index=False)

print("✅ Emotion labeling for the last 400k rows (with chunking) complete and saved!")

Device set to use cuda:0


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
import pandas as pd
from transformers import pipeline, AutoTokenizer
from tqdm import tqdm
import torch
import os

# ✅ Set CUDA to block for debugging if needed
# Uncomment the line below if debugging the CUDA error
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# ✅ Load dataset (last 400,000 rows)
df = pd.read_csv("/content/drive/MyDrive/converted_dataset.csv")
df_last_chunk = df.tail(100000).copy()

# ✅ Load model and tokenizer
model_name = "j-hartmann/emotion-english-distilroberta-base"
# Force CPU by setting device=-1
emotion_classifier = pipeline("text-classification", model=model_name, top_k=1, device=-1)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Function to split long texts by tokens (not words) into <=512 tokens
def split_text_by_tokens(text, max_tokens=512):
    tokens = tokenizer.encode(text, truncation=False)
    chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk, skip_special_tokens=True) for chunk in chunks]

# ✅ Emotion classification with chunking
def get_emotion(text):
    try:
        # If text is too long, split it
        chunks = split_text_by_tokens(text)
        emotions = []
        for chunk in chunks:
            prediction = emotion_classifier(chunk)
            emotions.append(prediction[0][0]['label'])  # Take the top label per chunk
        # Return the most frequent predicted emotion across all chunks
        return max(set(emotions), key=emotions.count)
    except Exception as e:
        print(f"⚠️ Error processing text: {e}")
        return "unknown"

# ✅ Apply the function with progress bar
tqdm.pandas(desc="🔄 Processing emotions (chunk safe)")
df_last_chunk['tag'] = df_last_chunk['pattern'].progress_apply(get_emotion)

# ✅ Save updated dataset
df_last_chunk.to_csv("/content/drive/MyDrive/converted_dataset_last_100k_with_tags.csv", index=False)
print("✅ Emotion labeling complete and saved!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

⚠️ Error processing text: index out of range in self


🔄 Processing emotions (chunk safe): 100%|█████████▉| 99711/100000 [3:58:28<00:32,  8.78it/s]

⚠️ Error processing text: index out of range in self


🔄 Processing emotions (chunk safe): 100%|██████████| 100000/100000 [3:59:08<00:00,  6.97it/s]


✅ Emotion labeling complete and saved!


In [ ]:
import pandas as pd
from transformers import pipeline, AutoTokenizer
from tqdm import tqdm

# ✅ Load the dataset
df = pd.read_csv("/content/drive/MyDrive/converted_dataset.csv")

# ✅ Select 300k rows from -400k to -100k (excluding the last 100k)
df_last_300k = df.iloc[-400000:most-100000].copy()

# ✅ Load the emotion classifier model and tokenizer
model_name = "j-hartmann/emotion-english-distilroberta-base"
emotion_classifier = pipeline("text-classification", model=model_name, top_k=1)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Function to split text into 512-token chunks
def split_text_by_tokens(text, max_length=512):
    tokens = tokenizer.encode(text, add_special_tokens=False)
    return [tokens[i:i + max_length] for i in range(0, len(tokens), max_length)]

# ✅ Emotion classification function with chunk handling
def get_emotion(text):
    if not isinstance(text, str) or not text.strip():
        return "neutral"  # Default for empty text

    token_chunks = split_text_by_tokens(text, max_length=512)
    emotions = []
    for chunk in token_chunks:
        chunk_text = tokenizer.decode(chunk, skip_special_tokens=True)
        emotion = emotion_classifier(chunk_text)
        emotions.append(emotion[0][0]['label'])
    return max(set(emotions), key=emotions.count)

# ✅ Batch processing: 10k rows per file with sequential saving
batch_size = 10000
total_batches = len(df_last_300k) // batch_size

for batch_num in range(total_batches):
    start_idx = batch_num * batch_size
    end_idx = start_idx + batch_size
    print(f"🔄 Processing batch {batch_num + 1}/{total_batches} (rows {start_idx} to {end_idx})...")

    df_batch = df_last_300k.iloc[start_idx:end_idx].copy()
    tqdm.pandas(desc=f"🔄 Batch {batch_num + 1}/{total_batches}")
    df_batch['tag'] = df_batch['pattern'].progress_apply(get_emotion)

    # ✅ Save each batch as a separate file in sequence
    output_path = f"/content/drive/MyDrive/converted_dataset_batch_{batch_num + 1}.csv"
    df_batch.to_csv(output_path, index=False)

    print(f"✅ Batch {batch_num + 1} saved as {output_path}\n")

print("🎉✅ All 300k rows processed in 10k batches and saved sequentially!")


Device set to use cuda:0


✅ Emotion labeling complete and saved!


In [ ]:
import pandas as pd

# ✅ File paths for the two CSV files
file1 = "/content/drive/MyDrive/file1.csv"
file2 = "/content/drive/MyDrive/file2.csv"

# ✅ Output file path for the combined CSV
output_file = "/content/drive/MyDrive/combined_dataset.csv"

# ✅ Load the two CSV files
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# ✅ Combine the two DataFrames
combined_df = pd.concat([df1, df2], ignore_index=True)

# ✅ Save the combined DataFrame to a new CSV file
combined_df.to_csv(output_file, index=False)

print(f"🎉✅ Files combined successfully and saved to: {output_file}")

# ✅ Optional: Preview combined data
print(f"📊 Combined dataset shape: {combined_df.shape}")
print(combined_df.head())

In [ ]:
import pandas as pd
import os

# ✅ Directory where your CSV files are stored
directory = "/content/drive/MyDrive/"

# ✅ Output file path for the combined CSV
output_file = os.path.join(directory, "combined_converted_dataset_300k.csv")

# ✅ List to store DataFrames
dataframes = []

# ✅ Loop through files in sequence and append to the list
for i in range(1, 31):  # Assuming we have 30 batch files
    file_path = os.path.join(directory, f"converted_dataset_batch_{i}.csv")
    if os.path.exists(file_path):
        print(f"🔄 Reading {file_path}...")
        df = pd.read_csv(file_path)
        dataframes.append(df)
    else:
        print(f"⚠️ File not found: {file_path}")

# ✅ Concatenate all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# ✅ Save the combined DataFrame
combined_df.to_csv(output_file, index=False)
print(f"🎉✅ All batches combined and saved to: {output_file}")

# ✅ Optional: Show the shape of the combined dataset
print(f"📊 Combined dataset shape: {combined_df.shape}")


In [ ]:
import pandas as pd

# ✅ File path for the CSV file
file_path = "/content/drive/MyDrive/correct_dataset.csv"

# ✅ Output file path for the sorted CSV
output_file = "/content/drive/MyDrive/correct_dataset_sorted.csv"

# ✅ Load the CSV file
df = pd.read_csv(file_path)

# ✅ Sort the DataFrame by the 'tag' column
df_sorted = df.sort_values(by='tag').reset_index(drop=True)

# ✅ Save the sorted DataFrame to a new CSV file
df_sorted.to_csv(output_file, index=False)

print(f"🎉✅ Dataset sorted by 'tag' column and saved to: {output_file}")

# ✅ Optional: Preview sorted data
print(f"📊 Sorted dataset shape: {df_sorted.shape}")
print(df_sorted.head())


In [ ]:
csv_file = "/content/converted_dataset_with_tags.csv"
df = pd.read_csv(csv_file)

df.head()

,pattern,response,tag
0,I've been feeling so sad and overwhelmed latel...,"Hey there, I'm here to listen and support you....",surprise
1,"I recently got a promotion at work, which I th...",I can understand how it can be overwhelming wh...,joy
2,"Well, the workload has increased significantly...",It sounds like you're dealing with a lot of pr...,sadness
3,I've been trying to prioritize my tasks and de...,It's great to hear that you're already impleme...,neutral
4,You're right. I haven't really opened up about...,"It's completely normal to feel that way, but r...",fear
